In [11]:
import os
import glob

exp_path = "/research/d5/gds/yzhong22/experiments/med_vlm_benchmark"
exp_path_mirror = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark"

In [1]:
# train

tune_modules = ["ML"]
datasets = {
    "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    # "PathVQA": "None",
    # "VQA-RAD": "None",
    # "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_train.py \
    --peft lora --lora_r 16 --lora_alpha 32 \
    --task vqa --dataset {} \
    --model MedGemma\
    --image_path {} \
    --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it \
    --bf16 True \
    --output_dir /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --num_train_epochs 3 \
    --per_device_train_batch_size {} \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps {} \
    --save_strategy "epoch" \
    --learning_rate 2e-4 \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "linear" \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --tune_modules {}"""

per_device_train_batch_size = 4
gradient_accumulation_steps = 4

assert per_device_train_batch_size * gradient_accumulation_steps == 16

commands = []

for data, path in datasets.items():
    for tm in tune_modules:
        commands.append(
            base_command.format(
                data,
                path,
                per_device_train_batch_size,
                int(gradient_accumulation_steps),
                tm,
            )
        )

print("&&".join(commands))

python run_train.py     --peft lora --lora_r 16 --lora_alpha 32     --task vqa --dataset SLAKE     --model MedGemma    --image_path /research/d5/gds/yzhong22/datasets/SLAKE/imgs     --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --bf16 True     --output_dir /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --num_train_epochs 3     --per_device_train_batch_size 4     --per_device_eval_batch_size 4     --gradient_accumulation_steps 4     --save_strategy "epoch"     --learning_rate 2e-4     --warmup_ratio 0.03     --lr_scheduler_type "linear"     --gradient_checkpointing True     --dataloader_num_workers 4     --tune_modules ML


# EVAL

## zero-shot

In [1]:
# eval

datasets = {
    "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_eval.py \
    --task vqa --dataset {} --split test \
    --image_path {} \
    --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it \
    --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --save_pred"""

commands = []
for data, path in datasets.items():

    commands.append(base_command.format(data, path))

print("&&".join(commands))

# print(commands[3])

python run_eval.py     --task vqa --dataset SLAKE --split test     --image_path /research/d5/gds/yzhong22/datasets/SLAKE/imgs     --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset PathVQA --split test     --image_path None     --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset VQA-RAD --split test     --image_path None     --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&pyth

In [ ]:
# zero-shot eval